In [3]:
# pip install snowflake-connector-python
# pip install snowflake-sqlalchemy
# pip install pyspark findspark

import os
# import findspark
# findspark.init()
from dotenv import load_dotenv
from pyspark.sql import SparkSession
load_dotenv()

# Initialize a Spark session
spark = (SparkSession.builder
    .appName("Snowflake to PySpark")
    .config("spark.jars.packages", "net.snowflake:spark-snowflake_2.12:2.10.0-spark_3.2") 
    .master("local[*]")
    .getOrCreate())

# Display the Spark session 
spark

In [8]:
# Snowflake connection options
sfOptions = {
    "sfURL": f"https://{os.getenv('ACCOUNT')}.snowflakecomputing.com",
    "sfUser": os.getenv('USER_NAME'),
    "sfPassword": os.getenv('PASSWORD'),
    "sfDatabase": "US_ZIP_CODE_CROSSWALK_HUDUPS",
    "sfSchema": "zipcode_crosswalk",
    "sfWarehouse": "compute_wh",
    "sfRole": "ACCOUNTADMIN"
}

# Read the data from Snowflake table into a DataFrame
df = spark.read \
    .format("net.snowflake.spark.snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "COVID19_EPIDEMIOLOGICAL_DATA.PUBLIC.DEMOGRAPHICS") \
    .load()

# Register the DataFrame as a temporary SQL view for processing
df.createOrReplaceTempView("demographics")


24/08/10 15:55:43 WARN SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.0.1 with a connector designed to support Spark 3.2. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.


In [9]:
df.show()

+---------+---------+-----+----------+------------+-----+-----------------+----------------+---------------------+-----------------------+
|ISO3166_1|ISO3166_2| FIPS|  LATITUDE|   LONGITUDE|STATE|           COUNTY|TOTAL_POPULATION|TOTAL_MALE_POPULATION|TOTAL_FEMALE_POPULATION|
+---------+---------+-----+----------+------------+-----+-----------------+----------------+---------------------+-----------------------+
|       US|       TX|48203|32.5036285| -94.0554582|   TX|  Harrison County|           66431|                32573|                  33858|
|       US|       TX|48205|35.7167664|-102.3805857|   TX|   Hartley County|            5966|                 3596|                   2370|
|       US|       TX|48207| 33.120371| -99.6317724|   TX|   Haskell County|            5812|                 3236|                   2576|
|       US|       TX|48209|30.0241697| -97.7473311|   TX|      Hays County|          185686|                92221|                  93465|
|       US|       TX|48211|

24/08/10 15:56:21 ERROR RestRequest: Stop retrying since elapsed time due to network issues has reached timeout. Elapsed: 62,871(ms), timeout: 60,000(ms)
24/08/10 15:57:27 ERROR RestRequest: Stop retrying since elapsed time due to network issues has reached timeout. Elapsed: 65,768(ms), timeout: 60,000(ms)
